# Food synset entities


In [1]:
import myfunctions as f
f.today()

'20180208'

In [2]:
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd()  

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


## Food synset attributes
>From delivery sites:

    1. food/index
    2. food/food_name 		assigned name to the food item
    3. food/synonyms	 	synonyms of the food item
    4. food/menu_names 		name variations at different restaurants


> From reviews:

    1. food/review					locations of storage of the review
    2. food/review.text				review containing the food item 
    3. food/review.clean_text	    processed review containing the food item
    4. food/review.restaurant       restaurant index	    
    5. food/review.image     	    image in review

### From delivery records

In [8]:
file_name = "all_food_vendor__standard__20180207.p"
df_retrived = f.retrive_file(file_name)

retrived: all_food_vendor__standard__20180207.p


In [10]:
print ("Got %d food-vendor records" % df_retrived.shape[0])
print ("Got %d unique food items " % df_retrived["standard_name"].nunique())
print df_retrived.columns.tolist()

Got 333572 food-vendor records
Got 92298 unique food items 
['food_name', 'vendor_name', 'locs', 'clean_name', 'lemma_name', 'lemma_name_2', 'standard_name']


In [39]:
# selct columns for delivery records
cols = ['standard_name', 'clean_name','food_name', 'vendor_name', 'locs', ]
df = df_retrived[cols]
# rename as cols in food synset
col_names = ["food_name", "synonyms", "menu_names", "restaurant", "locs",  ]
df.columns = col_names

# merge records based on food_name
tup = lambda g: tuple(g)
unq = lambda g:  sorted(list(set(g)))
enlist = lambda g:  [item[1] for sublist in g for item in sublist]
fn = { "synonyms":unq, "menu_names":tup, "restaurant":unq, "locs":enlist}
de2 = df.groupby("food_name").agg(fn)
de2.reset_index(inplace=True)
de2["menu_names"] = de2["menu_names"].apply(lambda food_lst: 
                                            sorted([(x,food_lst.count(x)) for x in set(food_lst)
                                                   ], key=lambda x: (-x[1], x)))
de2.reset_index(inplace=True)
de2["index"] = de2["index"].apply(lambda s: "food_"+ str(s))

# map restaurants to their respective index, for restaurants from delivery sites only
file_name = "restaurant_entities__20180207.p"
restaurant_entities = f.retrive_file(file_name)
restaurant_entities["type"] = restaurant_entities["index"].apply(lambda s: s.split("_")[0])
restaurant_entities = restaurant_entities[restaurant_entities["type"]=="delivery"]
restaurant_index_ref = restaurant_entities.set_index("restaurant.name").to_dict()["index"]
de2["restaurant"] = de2["restaurant"].apply(lambda lst: 
                                            sorted([restaurant_index_ref[r] for r in lst]))

retrived: restaurant_entities__20180207.p


In [46]:
de2.head(2)

,index,food_name,locs,synonyms,menu_names,restaurant
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]"
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]"


In [37]:
print ("Got %d unique food items " % de2["index"].nunique())

Got 92298 unique food items 


### From review

In [47]:
review_path = "/home/yueliu/Desktop/workspace_yue"
file_name = "burpple__review__clean__20180205.p"    
file_name = os.path.join(review_path, file_name)
df_retrived = f.retrive_file(file_name)

import pandas as pd
pd.options.mode.chained_assignment = None 
cols = ['loc', 'title', 'body', 'restaurant.name']
rev = df_retrived[cols]
sep = "***____***"
rev["text"] = rev["title"] + sep + rev["body"]
rev.fillna("", inplace=True)
rev_loc_rest_ref = rev.set_index("loc").to_dict()["restaurant.name"]
rev_loc_rev_ref = rev.set_index("loc").to_dict()["text"]

retrived: /home/yueliu/Desktop/workspace_yue/burpple__review__clean__20180205.p


In [74]:
file_name = "review_with_synsets__20180207.p"
file_name = os.path.join(review_path, file_name)
loc_food = f.retrive_file(file_name)

retrived: /home/yueliu/Desktop/workspace_yue/review_with_synsets__20180207.p


In [81]:
# merge records based on food_name
unq = lambda g:  sorted(list(set(g)))
enlist = lambda g:  [item for sublist in g for item in sublist]
fn = { "standard_text":unq, "loc":enlist}
bu2 = loc_food.groupby("food_items").agg(fn)
bu2.reset_index(inplace=True)
bu2["review.restaurant"] = bu2["loc"].apply(lambda lst: [rev_loc_rest_ref[l] for l in lst])
bu2["review.text"] = bu2["loc"].apply(lambda lst: [rev_loc_rev_ref[l] for l in lst])
bu2["review.image"] = bu2["loc"].apply(lambda lst: [l.split("/")[-1] for l in lst])
# map restaurants to their respective index, for restaurants from burpple only
file_name = "restaurant_entities__20180207.p"
restaurant_entities = f.retrive_file(file_name)
restaurant_entities["type"] = restaurant_entities["index"].apply(lambda s: s.split("_")[0])
restaurant_entities = restaurant_entities[restaurant_entities["type"]=="burpple"]
restaurant_index_ref = restaurant_entities.set_index("restaurant.name").to_dict()["index"]
bu2["review.restaurant"] = bu2["review.restaurant"].apply(lambda lst: 
                                            sorted([restaurant_index_ref[r] for r in lst]))
bu2.rename(index=str, columns={"food_items": "food_name", 
                               "loc": "review", 
                               "standard_text": "review.clean_text"}, inplace=True)

retrived: restaurant_entities__20180207.p


In [96]:
bu2.head(2)

,food_name,review,review.clean_text,review.restaurant,review.text,review.image
0,-PRON- ann,[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,[burpple_12581],[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q]
1,-PRON- beef,"[burppleinitial/review/XwPDjO8V, burppleinitia...","[-PRON- beef, -PRON- beef can understand why p...","[burpple_0, burpple_10107, burpple_10138, burp...",[@juneleety and I finally used our beanstro vo...,"[XwPDjO8V, AWWNXiyw, 9xOY4Zr5, heKHuAj5, 1m1CD..."


### Join

In [97]:
food_synset = pd.merge(de2, bu2, how='left', on=['food_name'])
print food_synset.shape
print food_synset.columns.tolist()
food_synset.head(2)

(92298, 11)
['index', 'food_name', 'locs', 'synonyms', 'menu_names', 'restaurant', 'review', 'review.clean_text', 'review.restaurant', 'review.text', 'review.image']


,index,food_name,locs,synonyms,menu_names,restaurant,review,review.clean_text,review.restaurant,review.text,review.image
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]",[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,[burpple_12581],[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q]
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]",NaN,NaN,NaN,NaN,NaN


In [98]:
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food_synset, file_name)

saved: food_entities__20180208.p
